Парсинг магаза 
https://www.respublica.ru/authors/{author_id}



In [1]:
from multiprocessing import Pool
from multiprocessing.dummy import Pool as ThreadPool
import time
import requests
from lxml import etree,html as lhtml
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm.notebook import tqdm
import json
from lxml import etree, html as lhtml
from time import sleep

In [2]:
link = 'https://www.respublica.ru/authors/'
link_on_author = list()
with open('authors.txt') as f:
    for num, author in enumerate(f):
        author = re.sub(r'\n', '', author)
        link_on_author.append(link + author +'?page=' )

In [3]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    for _ in range(n_attempts):
        page = requests.get(url,**kwargs)
        if page.ok:
            return page
        time.sleep(t_sleep)
    print(url, page.status_code)

In [4]:
def links(avtor):
    page = get_page(avtor).text
    soup = BeautifulSoup(page, 'html.parser')
    link = list(map(lambda s:s['href'],soup.find_all('a', class_="rd-listing-product-item__image-wrapper")))
    link = ['https://www.respublica.ru'+ i for i in link]
    return link


In [5]:
def all_linkss(avtor):
    book_avtor = list()
    num = 1
    while True:
        page = links(avtor +str(num))
        book_avtor.extend(page)
        num = num + 1
        if len(page) == 0:
            break
    return book_avtor

In [6]:
%%time
# all_links = list()
# for author in tqdm(link_on_author):
#     all_links.extend( vse_sulki(author) )
with ThreadPool(processes=10) as pool:
    mass = pool.map(all_linkss, link_on_author)
pool.join()
all_links = [ element for row in mass for element in row ]
len(all_links)

Wall time: 29.4 s


2453

In [7]:
one_book=all_links[1]
one_book

'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/525197-king-s-burya-stoletiya-korol-na-vse-vremena-m-o'

In [8]:
def process_page(url):
    
#     page = requests.get(url).text
    page = get_page(url).text
    tree = lhtml.fromstring(page)
    ez={}
    
    #ID+
    
    ez["ID"] = tree.xpath('//span[@itemprop="sku"]')[0].text
    
    #URL+
    
    ez["URL"] = url
    
    #Название+
    
    ez["Название"] =  tree.xpath('//h1[@class="rd-page-product__title"]')[0].text
    
    #Автор+
    
    ez["Автор"] = ';'.join(tree.xpath('//a[@itemprop="brand"]//text()'))
    
    #Превью

    pr = tree.xpath('//a[@class="download-pdf"]//attribute::href')
    ez['Превью'] = 'https://www.respublica.ru'+pr[0] if len(pr) != 0 else None


    #Изображение
    
    izo = tree.xpath('//img[@class="rd-page-product__img"]//attribute::src')
    ez['Изображение'] = 'https://www.respublica.ru'+izo[0]
    
    #Описание?
    
    op = tree.xpath('//div[@class="rd-page-product__desc-body"]//text()')
    op = '\n'.join(op)
    ez['Описание'] = re.sub(r'\s+', ' ', op.strip())
    
    #Цена
    
    ts = tree.xpath('//span[@class="num"]')[0].text
    ez['Цена'] = re.sub(r'\D','',ts)

    #Цена (старая)
    
    tss = tree.xpath('//span[@class="prev"]')
    ez["Цена (старая)"] = re.sub(r'\D', '', tss[0].text) if len(tss) != 0 else None

    #В наличии
    
    dn = tree.xpath('//span[@class="current"]')
    ez['В наличии'] = True if len(dn) != 0 else False

    #Категория
    
    ctg=tree.xpath('//span[@class="rd-page-breadcrumbs-item"]//span[@itemprop="name"]')
    c = ';'.join(ct.text for ct in ctg)
    ez['Категория'] = c
    
    #Блок 1
    blok1 = tree.xpath('//div[@class="rd-rating-stars"]')[0]
    #Число отзывов
    
    count_ot = blok1.xpath('.//attribute::content')
    ez['Число отзывов'] = count_ot[0] if len(count_ot) != 0 else None

    #Число оценок
    
    count_otsen = blok1.xpath('.//attribute::content')
    ez['Число оценок'] = count_otsen[1] if len(count_otsen) != 0 else None

    #Оценка
    
    otsen = blok1.xpath('.//attribute::content')
    ez['Оценка'] = otsen[3] if len(otsen) !=0 else None

    #Блок 2
    
    blok2 = tree.xpath('//div[@class="rd-page-product__desc-params"]')[0]
    
    #ТАБЛИЧКА
    
    name = blok2.xpath('.//p/*[contains(@itemprop, "name")]/text()')
    value = blok2.xpath('.//p/*[contains(@itemprop, "value")]/text()')
    ez.update(zip(name, value))
    return ez

In [9]:
process_page("https://www.respublica.ru/knigi/gumanitarnye-i-obschestvennye-nauki/filologiya/444113-pishi-sokraschay-kak-sozdavat-silnye-teksty")

{'ID': '444113',
 'URL': 'https://www.respublica.ru/knigi/gumanitarnye-i-obschestvennye-nauki/filologiya/444113-pishi-sokraschay-kak-sozdavat-silnye-teksty',
 'Название': 'Пиши, сокращай. Как создавать сильные тексты',
 'Автор': 'Максим Ильяхов',
 'Превью': 'https://www.respublica.ru/items/319412/download_preview_pdf',
 'Изображение': 'https://www.respublica.ru/uploads/01/00/00/6e/ov/large_888e55eb57676e2d.jpg',
 'Описание': 'Авторы на конкретных примерах показывают, что такое хорошо и что такое плохо в информационных, рекламных, журналистских и публицистических текстах. Как писать письма, на которые будут отвечать, и рассылки, от которых не будут отписываться. Как создавать действенные и не вульгарные рекламные объявления. Как излагать мысли кратко, ясно и убедительно: без языкового мусора, фальши и штампов. Следуя рекомендациям в книге, вы научитесь писать понятно, увлекать читателей и добиваться доверия. Это обязательная книга для копирайтеров, авторов и редакторов, а также дизайнер

In [10]:
process_page(all_links[3])

{'ID': '513925',
 'URL': 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/fantastika-i-fentezi/513925-king-s-chuzhak-temnaya-bashnya-t-o',
 'Название': 'Чужак',
 'Автор': 'Стивен Кинг',
 'Превью': 'https://www.respublica.ru/items/373815/download_preview_pdf',
 'Изображение': 'https://www.respublica.ru/uploads/00/00/00/bz/ca/large_1c84b5fe6ccdd1e1.jpg',
 'Описание': 'В парке маленького городка Флинт-Сити найден труп жестоко убитого одиннадцатилетнего мальчика. Все улики, показания свидетелей указывают на одного человека — Терри Мейтленда. Тренер молодежной бейсбольной команды, преподаватель английского, муж и отец двух дочерей — неужели он был способен на такое?К тому же у Терри есть неопровержимое алиби: на момент совершения преступления он был в другом городе.Но как мог один и тот же человек оказаться в двух местах одновременно? Или в городе появилось НЕЧТО, способное принимать обличие любого человека?..Детектив полиции Флинт-Сити Ральф Андерсон и частный сыщик агентства "

In [11]:
process_page('https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/178271-kot-bez-prikras')

{'ID': '178271',
 'URL': 'https://www.respublica.ru/knigi/hudozhestvennaya-literatura/sovremennaya-proza/178271-kot-bez-prikras',
 'Название': 'Кот без прикрас',
 'Автор': 'Терри Пратчетт',
 'Превью': None,
 'Изображение': 'https://www.respublica.ru/uploads/01/00/00/4p/sf/large_3cabef337cd36346.jpg',
 'Описание': 'Вначале было слово, и это слово было Кот. Эту неколебимую истину возвестил народам кошачий бог через послушного своего ученика Терри Пратчетта. Итак, перед вами евангелие породы кошачьих. Вы узнаете о котах все. Из чего коты сделаны, их внутреннее устройство, законы, которые на них действуют и не действуют, что они едят и что пьют. Ну а чтобы было наглядней, книга снабжена картинками замечательного английского художника-графика Грея Джоллиффа. Да пребудет с вами кошачье благословение! Мяу!',
 'Цена': '370',
 'Цена (старая)': None,
 'В наличии': False,
 'Категория': ' Книги; Художественная литература; Современная проза',
 'Число отзывов': None,
 'Число оценок': None,
 'Оценка'

In [12]:
from multiprocessing.dummy import Pool, Lock, Value

mutex = Lock()
n_processed = Value('i',0)
def func_wrapper(uid):
    res = process_page(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

In [13]:
# result = list(map(process_page, all_links[:10]))
with ThreadPool(processes=10) as pool:
    result = pool.map(func_wrapper, all_links)
pool.join()
df = pd.DataFrame(result)
df.sort_values(by=['ID'], inplace=True)
df = df.fillna('')
with open('hw_3.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

2450 objects are processed...

In [15]:
pd.set_option('display.max_columns', 100)
print(df.shape)
df.head(10)

(2453, 42)


,ID,URL,Название,Автор,Превью,Изображение,Описание,Цена,Цена (старая),В наличии,Категория,Число отзывов,Число оценок,Оценка,ISBN,Издательство,Серия,Обложка,Формат,Количество страниц,Год издания,Язык,"Вес, г",Раздел,Направление,Возрастные ограничения,Страна-производитель,Жанр,Герои,Эпоха,Возраст,Иллюстратор,Тип,Вид бумаги,Материал,"Размер, см",Пол,Назначение,Тематика,Упаковка,Рисунок,Иллюстрации
2062,000033,https://www.respublica.ru/knigi/hudozhestvenna...,Много шума из ничего,Уильям Шекспир,,https://www.respublica.ru/uploads/00/00/00/cq/...,В настоящем издании под одной обложкой объедин...,100,140,False,Книги; Художественная литература; Поэзия,,,,978-5-389-03203-3,Азбука,,Мягкая,11 х 18,224,2015,Русский,,Зарубежные,,,Россия,,,,,,,,,,,,,,,
2061,000056,https://www.respublica.ru/knigi/hudozhestvenna...,Двенадцатая ночь,Уильям Шекспир,,https://www.respublica.ru/uploads/01/00/00/24/...,Перу великого английского драматурга Уильяма Ш...,100,140,False,Книги; Художественная литература; Поэзия,,,,978-5-389-03202-6,Азбука,,Мягкая,11 х 18,192,2011,Русский,,Зарубежные,,,Россия,,,,,,,,,,,,,,,
1846,021678,https://www.respublica.ru/knigi/hudozhestvenna...,N or M?,Agatha Christie,,https://www.respublica.ru/uploads/00/00/00/68/...,Tommy & Tuppence are hired to track down warti...,610,,False,Книги; Художественная литература; Книги на ин...,,,,,HarperCollins,,Мягкая,10 х 17,304,2001,Английский,,Зарубежные,,,США,Детектив,,,,,,,,,,,,,,
1843,021679,https://www.respublica.ru/knigi/hudozhestvenna...,The Secret Adversary,Agatha Christie,,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie's first Tommy and Tuppence mys...,610,,False,Книги; Художественная литература; Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,400,2011,Английский,,Зарубежные,,,США,Детектив,,,,,,,,,,,,,,
1870,021680,https://www.respublica.ru/knigi/hudozhestvenna...,By the Pricking of My Thumbs,Agatha Christie,,https://www.respublica.ru/uploads/01/00/00/68/...,An old woman in a nursing home speaks of a chi...,510,610,False,Книги; Художественная литература; Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,352,2011,Английский,,Зарубежные,,,США,Детектив,,,,,,,,,,,,,,
1847,021681,https://www.respublica.ru/knigi/hudozhestvenna...,Partners in Crime,Agatha Christie,,https://www.respublica.ru/uploads/00/00/00/68/...,Agatha Christie's complete Tommy and Tuppence ...,610,,False,Книги; Художественная литература; Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,352,2001,Английский,,Зарубежные,,,США,Детектив,,,,,,,,,,,,,,
1836,021700,https://www.respublica.ru/knigi/hudozhestvenna...,Evil Under the Sun,Agatha Christie,,https://www.respublica.ru/uploads/00/00/00/0p/...,The beautiful bronzed body of Arlena Stuart la...,540,,False,Книги; Художественная литература; Книги на ин...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1853,021702,https://www.respublica.ru/knigi/hudozhestvenna...,The Murder on the Links,Agatha Christie,,https://www.respublica.ru/uploads/01/00/00/68/...,"On a French golf course, a millionaire is foun...",639,,False,Книги; Художественная литература; Книги на ин...,,,,,HarperCollins,,Мягкая,13 x 20,288,2017,,,,,,США,,,,,,,,,,,,,,,
1892,021703,https://www.respublica.ru/knigi/hudozhestvenna...,The ABC Murders,Agatha Christie,,https://www.respublica.ru/uploads/01/00/00/5d/...,"There's a serial killer on the loose, working ...",610,,False,Книги; Художественная литература; Книги на ин...,,,,,HarperCollins,,Мягкая,11 х 17,278,2010,Английский,,Зарубежные,,,США,Детектив,,,,,,,,,,,,,,
1877,021706,https://www.respublica.ru/knigi/hudozhestvenna...,Death on the Nile,Agatha Christie,,https://www.respublica.ru/uploads/01/00/00/60/...,The tranquillity of a cruise along the Nile wa...,490,,False,Книги; Художественная литература; Книги на ин...,,,,978-0-00-711932-5,HarperCollins,Poirot,Мягкая,11 х 17,416,2001,Английский,,Зарубежные,,,США,Детектив,,,,,,,,,,,,,,
